In [ ]:
# ! ln ../bcc/inputs/163_A1*.npy inputs
# ! ln ../bcc/gnn_models/4.model.pth models/tumor_map_gnn.pth
# ! ln ../bcc/pretrain_model.pth models/tumor_map_cnn.pth
# ! ln ../fat_dermis_epi_sq_model/v2/checkpoints_tissue_seg/104.checkpoint.pth models/macro_map_cnn.pth
# ! ln ../nuclei_pipeline/seg_model/27.checkpoint.pth models/nuclei.pth
# ! pip uninstall pathpretrain -y && pip install git+https://github.com/jlevy44/PathPretrain

In [ ]:
# preprocess

In [46]:
import os, tqdm
import numpy as np, pandas as pd
from pathflowai.utils import generate_tissue_mask
from itertools import product
from scipy.ndimage.morphology import binary_fill_holes as fill_holes

def preprocess(basename="163_A1a",
               threshold=0.05,
               patch_size=256):
    
    image=f"inputs/{basename}.npy"
    basename=os.path.basename(image).replace('.npy','')
    image=np.load(image)
    
    masks=dict()
    masks['tumor_map']=generate_tissue_mask(image,
                             compression=10,
                             otsu=False,
                             threshold=240,
                             connectivity=8,
                             kernel=5,
                             min_object_size=100000,
                             return_convex_hull=False,
                             keep_holes=False,
                             max_hole_size=6000,
                             gray_before_close=True,
                             blur_size=51) 
    x_max,y_max=mask.shape
    masks['macro_map']=fill_holes(masks['tumor_map'])
    
    patch_info=dict()
    for k in masks:
        patch_info[k]=pd.DataFrame([[basename,x,y,patch_size,"0"] for x,y in tqdm.tqdm(list(product(range(0,x_max-patch_size,patch_size),range(0,y_max-patch_size,patch_size))))],columns=['ID','x','y','patch_size','annotation'])
        patches=np.stack([image[x:x+patch_size,y:y+patch_size] for x,y in tqdm.tqdm(patch_info[k][['x','y']].values.tolist())])                   
        include_patches=np.stack([masks[k][x:x+patch_size,y:y+patch_size] for x,y in tqdm.tqdm(patch_info[k][['x','y']].values.tolist())]).mean((1,2))>=threshold

        np.save(f"masks/{basename}_{k}.npy",masks[k])
        np.save(f"patches/{basename}_{k}.npy",patches[include_patches]) 
        patch_info[k].iloc[include_patches].to_pickle(f"patches/{basename}_{k}.pkl")



In [50]:
# test component

In [2]:
# predict model
import os, torch, tqdm, pandas as pd, numpy as np
from torch.utils.data import Dataset, DataLoader 
from PIL import Image
from pathpretrain.train_model import train_model, generate_transformers, generate_kornia_transforms

class CustomDataset(Dataset):
    # load using saved patches and mask file
    def __init__(self, patch_info, npy_file, transform):
        self.X=np.load(npy_file)
        self.patch_info=pd.read_pickle(patch_info)
        self.xy=self.patch_info[['x','y']].values
        self.patch_size=self.patch_info['patch_size'].iloc[0]
        self.length=self.patch_info.shape[0]
        self.transform=transform
        self.to_pil=lambda x: Image.fromarray(x)
        self.ID=os.path.basename(npy_file).replace(".npy","")
        
    def __getitem__(self,i):
        x,y=self.xy[i]
        return self.transform(self.to_pil(self.X[i]))#[x:x+patch_size,y:y+patch_size]
        
    def __len__(self):
        return self.length
    
    def embed(self,model,batch_size,out_dir):
        Z=[]
        dataloader=DataLoader(self,batch_size=batch_size,shuffle=False)
        n_batches=len(self)//batch_size
        with torch.no_grad():
            for i,X in tqdm.tqdm(enumerate(dataloader),total=n_batches):
                if torch.cuda.is_available(): X=X.cuda()
                z=model(X).detach().cpu().numpy()
                Z.append(z)
        Z=np.vstack(Z)
        torch.save(dict(embeddings=Z,patch_info=self.patch_info),os.path.join(out_dir,f"{self.ID}.pkl"))
        
def generate_embeddings(basename="163_A1a",
                        analysis_type="tumor",
                       gpu_id=0):
    patch_info_file,npy_file=f"patches/{basename}_{analysis_type}_map.pkl",f"patches/{basename}_{analysis_type}_map.npy"
    models={k:f"models/{k}_map_cnn.pth" for k in ['macro','tumor']}
    num_classes=dict(macro=4,tumor=2)
    train_model(model_save_loc=models[analysis_type],extract_embeddings=True,num_classes=num_classes[analysis_type],predict=True,embedding_out_dir="cnn_embeddings/",custom_dataset=CustomDataset(patch_info_file,npy_file,generate_transformers(224,256)['test']),gpu_id=gpu_id)



In [ ]:
# create graph

In [8]:
import os, torch, numpy as np, pandas as pd
import pickle
import scipy.sparse as sps
from torch_geometric.utils import subgraph, add_remaining_self_loops
from torch_cluster import radius_graph
from collections import Counter
from torch_geometric.data import Data 

def create_graph_data(basename="163_A1a",
                      analysis_type="tumor",
                      radius=256,
                      min_component_size=600):
    embeddings=torch.load(f"cnn_embeddings/{basename}_{analysis_type}_map.pkl")
    xy=torch.tensor(embeddings['patch_info'][['x','y']].values).float().cuda()
    X=torch.tensor(embeddings['embeddings'])
    G=radius_graph(xy, r=radius*np.sqrt(2), batch=None, loop=True)
    G=G.detach().cpu()
    G=add_remaining_self_loops(G)[0]
    xy=xy.detach().cpu()
    datasets=[]
    edges=G.detach().cpu().numpy().astype(int)
    n_components,components=list(sps.csgraph.connected_components(sps.coo_matrix((np.ones_like(edges[0]),(edges[0],edges[1])))))
    comp_count=Counter(components)
    components=torch.LongTensor(components)
    for i in range(n_components):
        if comp_count[i]>=min_component_size:
            G_new=subgraph(components==i,G,relabel_nodes=True)[0]
            xy_new=xy[components==i]
            X_new=X[components==i]
            np.random.seed(42)
            idx=np.arange(X_new.shape[0])
            idx2=np.arange(X_new.shape[0])
            np.random.shuffle(idx)
            train_idx,val_idx,test_idx=torch.tensor(np.isin(idx2,idx[:int(0.8*len(idx))])),torch.tensor(np.isin(idx2,idx[int(0.8*len(idx)):int(0.9*len(idx))])),torch.tensor(np.isin(idx2,idx[int(0.9*len(idx)):]))
            dataset=Data(x=X_new, edge_index=G_new, y_new=torch.ones(len(X_new)), edge_attr=None, pos=xy_new)
            dataset.train_mask=train_idx
            dataset.val_mask=val_idx
            dataset.test_mask=test_idx
            dataset.id=basename
            dataset.component=i
            datasets.append(dataset)
    pickle.dump(datasets,open(os.path.join('graph_datasets',f"{basename}_{analysis_type}_map.pkl"),'wb'))

In [11]:
# predict graph

In [18]:
import os, torch, pickle, numpy as np, pandas as pd, torch.nn as nn
from torch_geometric.data import DataLoader
from torch_geometric.utils import to_dense_batch, to_dense_adj, dense_to_sparse, dropout_adj, to_networkx
from torch_geometric.nn import GATConv
import torch.nn.functional as F

class GCNNet(torch.nn.Module):
    def __init__(self, inp_dim, out_dim, hidden_topology=[32,64,128,128], p=0.5, p2=0.1, drop_each=True):
        super(GCNNet, self).__init__()
        self.out_dim=out_dim
        self.convs = nn.ModuleList([GATConv(inp_dim, hidden_topology[0])]+[GATConv(hidden_topology[i],hidden_topology[i+1]) for i in range(len(hidden_topology[:-1]))])
        self.drop_edge = lambda edge_index: dropout_adj(edge_index,p=p2)[0]
        self.dropout = nn.Dropout(p)
        self.fc = nn.Linear(hidden_topology[-1], out_dim)
        self.drop_each=drop_each

    def forward(self, x, edge_index, edge_attr=None):
        for conv in self.convs:
            if self.drop_each and self.training: edge_index=self.drop_edge(edge_index)
            x = F.relu(conv(x, edge_index, edge_attr))
        if self.training:
            x = self.dropout(x)
        x = self.fc(x)
        return x
    
class GCNFeatures(torch.nn.Module):
    def __init__(self, gcn, bayes=False, p=0.05, p2=0.1):
        super(GCNFeatures, self).__init__()
        self.gcn=gcn
        self.drop_each=bayes
        self.gcn.drop_edge = lambda edge_index: dropout_adj(edge_index,p=p2)[0]
        self.gcn.dropout = nn.Dropout(p)
    
    def forward(self, x, edge_index, edge_attr=None):
        for i,conv in enumerate(self.gcn.convs):
            if self.drop_each: edge_index=self.gcn.drop_edge(edge_index)
            x = conv(x, edge_index, edge_attr)
            if i+1<len(self.gcn.convs):
                x=F.relu(x)
        if self.drop_each:
            x = self.gcn.dropout(x)
        y = self.gcn.fc(F.relu(x))#F.softmax()
        return x,y
    
def predict(basename="163_A1a",
            analysis_type="tumor",
            gpu_id=0):
    hidden_topology=dict(tumor=[32]*3,macro=[32]*3)
    num_classes=dict(macro=4,tumor=2)
    torch.cuda.set_device(gpu_id)
    dataset=pickle.load(open(os.path.join('graph_datasets',f"{basename}_{analysis_type}_map.pkl"),'rb'))
    model=GCNNet(dataset[0].x.shape[1],num_classes[analysis_type],hidden_topology=hidden_topology[analysis_type],p=0.,p2=0.)
    model=model.cuda()
    model.load_state_dict(torch.load(os.path.join("models",f"{analysis_type}_map_gnn.pth"),map_location=f"cuda:{gpu_id}" if gpu_id>=0 else "cpu"))
    dataloader=DataLoader(dataset,shuffle=False,batch_size=1)
    model.eval()
    feature_extractor=GCNFeatures(model,bayes=False).cuda()
    graphs=[]
    for i,data in enumerate(dataloader):
        with torch.no_grad():
            graph = to_networkx(data).to_undirected()
            model.train(False)
            x=data.x.cuda()
            xy=data.pos.numpy()
            edge_index=data.edge_index.cuda()
            preds=feature_extractor(x,edge_index)
            z,y_pred=preds[0].detach().cpu().numpy(),preds[1].detach().cpu().numpy()
            graphs.append(dict(G=graph,xy=xy,z=z,y_pred=y_pred,slide=data.id,component=data.component))
    torch.save(graphs,os.path.join("gnn_results",f"{basename}_{analysis_type}_map.pkl"))


In [13]:
# nuclei prediction

In [2]:
from PIL import Image
from torch.utils.data import Dataset
import torch, pandas as pd, numpy as np
import pickle
from pathpretrain.train_model import train_model, generate_transformers, generate_kornia_transforms
from tqdm import trange

class WSI_Dataset(Dataset):
    def __init__(self, patches, transform):
        self.patches=patches
        self.to_pil=lambda x: Image.fromarray(x)
        self.length=len(self.patches)
        self.transform=transform
        
    def __getitem__(self,idx):
        X=self.transform(self.to_pil(self.patches[idx]))
        return X,torch.zeros(X.shape[-2:]).unsqueeze(0).long()
    
    def __len__(self):
        return self.length
    
def predict_nuclei(basename="163_A1a",
                   analysis_type="tumor",
                   gpu_id=0):
    patch_size=256
    patch_info_file,npy_file=f"patches/{basename}_{analysis_type}_map.pkl",f"patches/{basename}_{analysis_type}_map.npy"
    patches=np.load(npy_file)
    custom_dataset=WSI_Dataset(patches,generate_transformers(256,256)['test'])
    Y_seg=train_model(inputs_dir='inputs',
                    architecture='resnet50',
                    batch_size=512,
                    num_classes=2,
                    predict=True,
                    model_save_loc="models/nuclei.pth",
                    predictions_save_path='tmp_test.pkl',
                    predict_set='custom',
                    verbose=False,
                    class_balance=False,
                    gpu_id=gpu_id,
                    tensor_dataset=False,
                    semantic_segmentation=True,
                    custom_dataset=custom_dataset,
                    save_predictions=False)['pred']

    xy=pd.read_pickle(patch_info_file)[['x','y']].values
    img_shape=np.load(f"inputs/{basename}.npy",mmap_mode="r").shape[:-1]
    pred_mask=np.zeros(img_shape)
    for i in trange(Y_seg.shape[0]):
        x,y=xy[i]
        pred_mask[x:x+patch_size,y:y+patch_size]=Y_seg[i]
    pred_mask=pred_mask.astype(bool)
    np.save(f"nuclei_results/{basename}.npy",pred_mask)


In [6]:
! ls ../ArcticAI_Prototype/

0_5_pretrain_bcc.ipynb				   8_thumbnail_inspect.ipynb
0_generate_thumbnails_edit_graph_dataset.ipynb	   LICENSE
1_generate_graph_data.ipynb			   README.md
2_cv_splits.ipynb				   command_history
3_5_extract_features.ipynb			   complete_old
3_7_interpolate_label_expansion_apply_image.ipynb  extract_features.py
3_9_prepare_dzi.ipynb				   filters.py
3_train_gnn_model.ipynb				   ink
4_5_shape2.ipynb				   launch_feature_extraction.py
4_shape_analysis.ipynb				   modify_patch_db.ipynb
5_3d_align.ipynb				   nuclei_pipeline
6_ink_filter_practice.ipynb			   tissue_quality
7_align_slides.ipynb


In [ ]:
# estimate Mapper graphs macro+tumor

In [ ]:
# ink prediction

In [ ]:
# blend predicted scores

In [ ]:
# organize / cut sections

In [ ]:
# estimate alignment parameters

In [ ]:
# apply slide level alignment parameters to each exported image + mask
# original, macro, tumor, nuclei

In [ ]:
# build dzi

In [ ]:
# quality score

In [ ]:
# report generation

In [ ]:
# add more here; turn each cell into python script after testing
# arcticai package
# airflow scripts

In [ ]:
# OLD

def filter_mask(mask): # fill holes here (2 masks output) and ensure only have top X sections; find largest sections; do later
    macro_mask=fill_holes(mask)
    return mask, macro_mask

# holes for certain analysis type; rip from pathflow and lower dependencies
# preprocess
# analysis_type=""
# turn into custom dataset for pathpretrain eval
# predict(hidden_topology=[32]*3)
# predict_nuclei()